This code analyzes how the FRET Ratio (FRET YFP / CFP) changes over the course of a timecourse to explore the effect of Oligomycin and IAA treatment on the FRET Ratio.

Import Libraries

In [ ]:
import czifile
import numpy as np
from skimage.filters import gaussian, threshold_otsu
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt
import cv2
from IPython.display import Image, display
from skimage.color import label2rgb
from skimage.filters import gaussian, sobel
from skimage.measure import label, regionprops
from skimage.morphology import binary_erosion, binary_dilation, disk
from scipy.ndimage import binary_fill_holes
from matplotlib.colors import LogNorm
from skimage import exposure, filters, measure
import scipy.ndimage as ndi
import pandas as pd
import os

Define Sub Functions

In [ ]:

def load_czi_data(image_path, green_channel_idx, red_channel_idx):
    """Load and extract green and red channel data from a CZI file."""
    czi_file = czifile.CziFile(image_path)
    czi_data = czi_file.asarray()
    
    green_channel_data = np.squeeze(czi_data[:, :, green_channel_idx, :, :, :])
    red_channel_data = np.squeeze(czi_data[:, :, red_channel_idx, :, :, :])
    
    return green_channel_data, red_channel_data

def preprocess_image(channel_data, contrast_cutoff=0.4):
    """Preprocess the image by adjusting contrast and normalizing pixel values."""
    channel_data = exposure.adjust_sigmoid(channel_data, cutoff=contrast_cutoff)
    normalized_channel = (channel_data - channel_data.min()) / (channel_data.max() - channel_data.min())
    return normalized_channel

def segment_cells(green_channel_normalized):
    """Segment cells using thresholding, edge detection, and morphological operations."""
    threshold_value = threshold_otsu(green_channel_normalized)
    binary_image = green_channel_normalized > threshold_value

    edges = sobel(binary_image)  # Edge detection
    dilated_edges = binary_dilation(edges, disk(2))  # Increase border thickness
    eroded_edges = binary_erosion(dilated_edges, disk(1))  # Refine borders

    labeled_cells = label(eroded_edges)  # Label individual cells
    return labeled_cells

def analyze_region(labeled_image, region, green_channel, red_channel, frame_index, dir_name):
    """Analyze a specific cell region for intensity and MFI ratio."""
    if region.area < 1000:  # Skip small regions
        return None

    # Create masks for the region
    mask = labeled_image == region.label
    dilated_mask = binary_fill_holes(mask)
    inverted_mask = ~dilated_mask

    # Apply the mask to each channel
    channel1_masked = green_channel * inverted_mask
    channel2_masked = red_channel * inverted_mask

    # Calculate mean fluorescence intensity for each channel
    mean_intensity_channel1 = np.mean(channel1_masked[inverted_mask > 0])
    mean_intensity_channel2 = np.mean(channel2_masked[inverted_mask > 0])
    
    if mean_intensity_channel2 == 0:
        return None  # Skip if there's no signal in channel 2

    intensity_ratio = mean_intensity_channel1 / mean_intensity_channel2

    # Create MFI ratio map
    dilated_mask_region = dilated_mask[region.bbox[0]:region.bbox[2], region.bbox[1]:region.bbox[3]]
    channel1_masked_region = green_channel[region.bbox[0]:region.bbox[2], region.bbox[1]:region.bbox[3]] * dilated_mask_region
    channel2_masked_region = red_channel[region.bbox[0]:region.bbox[2], region.bbox[1]:region.bbox[3]] * dilated_mask_region

    with np.errstate(divide='ignore', invalid='ignore'):
        pixel_ratios = np.where(channel1_masked_region != 0, channel2_masked_region / channel1_masked_region, 0)

    mfi_ratio_map = np.zeros(region.image.shape, dtype=float)
    mfi_ratio_map[dilated_mask_region] = pixel_ratios[dilated_mask_region]

    visualize_mfi_ratio_map(mfi_ratio_map, region, frame_index, dir_name)

    return mean_intensity_channel1, mean_intensity_channel2, intensity_ratio

def visualize_mfi_ratio_map(mfi_ratio_map, region, frame_index, dir_name):
    """Visualize and save the MFI ratio map."""
    cmap = 'viridis'
    positive_mfi_ratio_map = mfi_ratio_map.copy()
    positive_mfi_ratio_map[positive_mfi_ratio_map <= 0] = np.nanmin(positive_mfi_ratio_map[positive_mfi_ratio_map > 0])
    
    norm = LogNorm(vmin=np.nanmin(positive_mfi_ratio_map), vmax=np.nanmax(positive_mfi_ratio_map))

    plt.imshow(positive_mfi_ratio_map, cmap=cmap, norm=norm)
    plt.colorbar(label='MFI Ratio (Log Scale)')
    plt.title(f'Per-Pixel MFI Ratio Map (Region {region.label})')
    plt.axis('off')
    
    output_path = os.path.join(dir_name, f'mfi_ratio_map_region{region.label}_{frame_index}.png')
    plt.savefig(output_path)
    plt.show()

def analyze_time_course_image(image_path, basename, dir_name):
    """Main function to analyze time-course images, including Oligomycin and IAA."""
    green_channel_data, red_channel_data = load_czi_data(image_path, green_channel_idx=1, red_channel_idx=0)
    
    results = []
    labeled_image = segment_cells(preprocess_image(green_channel_data[0, :, :]))

    cell_counter = 0
    basename_endings = ['UT', 'Oligomycin', 'IAA']

    for region in regionprops(labeled_image):
        cell_counter += 1

        for basename_ending in basename_endings:
            modified_image_path = image_path.replace('UT', basename_ending)
            green_channel_data_modified, red_channel_data_modified = load_czi_data(modified_image_path, green_channel_idx=1, red_channel_idx=0)

            for frame_index in range(green_channel_data_modified.shape[0]):
                result = analyze_region(labeled_image, region, green_channel_data_modified[frame_index, :, :], red_channel_data_modified[frame_index, :, :], frame_index, dir_name)
                if result:
                    mean_intensity_channel1, mean_intensity_channel2, intensity_ratio = result
                    results.append((basename, cell_counter, frame_index, mean_intensity_channel1, mean_intensity_channel2, intensity_ratio))

    return results

def analyze_timecourse(file_path, basename, dir_name):
    """Wrapper function to analyze a time course and return a DataFrame."""
    results = analyze_time_course_image(file_path, basename, dir_name)
    
    df = pd.DataFrame(results, columns=["Filename", "Cell", "Time", "CFP_MFI", "YFP_FRET_MFI", "Intensity_Ratio"])
    return df



Main Function

In [ ]:
def main():
    """Main execution function for processing all images in a folder."""
    folder_path = r"test_images"
    all_data_timecourse = []

    for well_image in os.listdir(folder_path):
        if well_image.lower().endswith(".czi"):  # Filter for CZI files
            well_image_path = os.path.join(folder_path, well_image)
            well_image_base_name = os.path.basename(well_image)[:-4]
            cwd = os.getcwd() #get current working directory
            well_image_dir = os.path.join(cwd, well_image_base_name)
            if "timecourse" and "UT" in well_image_base_name:
                os.makedirs(well_image_dir, exist_ok=True)
                df = analyze_timecourse(well_image_path, well_image_base_name, well_image_dir)
                all_data_timecourse.append(df)

    combined_df_timecourse = pd.concat(all_data_timecourse, ignore_index=True)
    combined_csv_path = os.path.join(cwd, "ATP_FRET_TIMECOURSE.xlsx")
    combined_df_timecourse.to_excel(combined_csv_path, index=False)

if __name__ == "__main__":
    main()